In [13]:
import pandas as pd
import numpy as np

### LOAD DATASET

In [14]:
# Load the data
file = "XRPDaily"
file_path = file + '.xlsx'  
data = pd.read_excel(file_path)

# Convert date to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data = data.sort_values('Date')

In [15]:
print(data.head(2))
print(data.tail(2))

        Date      Open      High       Low     Close     Volume
0 2017-11-09  0.217911  0.221791  0.214866  0.217488  147916992
1 2017-11-10  0.218256  0.219068  0.205260  0.206483  141032992
           Date      Open      High       Low     Close      Volume
2467 2024-08-11  0.585586  0.593077  0.547541  0.552932  1202704291
2468 2024-08-12  0.552931  0.579069  0.552163  0.568768  1479740403


### CALCULATE TECHNICAL INDICATORS


In [16]:
# Calculate RSI
def calculate_rsi(data, window):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

data['RSI_14'] = calculate_rsi(data, 14)

# Calculate Williams %R
def calculate_wr(data, period):
    highest_high = data['High'].rolling(window=period).max()
    lowest_low = data['Low'].rolling(window=period).min()
    wr = -100 * (highest_high - data['Close']) / (highest_high - lowest_low)
    return wr

data['Williams_%R_14'] = calculate_wr(data, 14)

# Calculate WMA
def calculate_wma(data, window):
    weights = np.arange(1, window + 1)
    wma = data['Close'].rolling(window).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return wma

data['WMA_10'] = calculate_wma(data, 10)

# Calculate EMA
data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()

# Calculate SMA
data['SMA_10'] = data['Close'].rolling(window=10).mean()

# Calculate HMA
def calculate_hma(data, window):
    half_length = int(window / 2)
    sqrt_length = int(np.sqrt(window))
    wmaf = calculate_wma(data, half_length)
    wmas = calculate_wma(data, window)
    raw_hma = 2 * wmaf - wmas
    hma = calculate_wma(pd.DataFrame(raw_hma), sqrt_length)
    return hma

data['HMA_10'] = calculate_hma(data, 10)

# Calculate Triple EMA
data['Triple_EMA_10'] = 3 * data['EMA_10'] - 3 * data['EMA_10'].ewm(span=10, adjust=False).mean() + data['EMA_10'].ewm(span=10, adjust=False).mean().ewm(span=10, adjust=False).mean()

# Calculate CCI
def calculate_cci(data, ndays): 
    TP = (data['High'] + data['Low'] + data['Close']) / 3 
    CCI = pd.Series((TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).std()), name = 'CCI') 
    return CCI

data['CCI_14'] = calculate_cci(data, 14)

# Calculate CMO
def calculate_cmo(data, period):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).sum()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).sum()
    cmo = 100 * (gain - loss) / (gain + loss)
    return cmo

data['CMO_14'] = calculate_cmo(data, 14)

# Calculate MACD
# btc_data['MACD'], btc_data['MACD_signal'] = btc_data['Close'].ewm(span=12, adjust=False).mean() - btc_data['Close'].ewm(span=26, adjust=False).mean(), btc_data['Close'].ewm(span=9, adjust=False).mean()
data['MACD'] = data['Close'].ewm(span=12, adjust=False).mean() - data['Close'].ewm(span=26, adjust=False).mean()



# Calculate PPO
def calculate_ppo(data, fast_period=12, slow_period=26):
    fast_ema = data['Close'].ewm(span=fast_period, adjust=False).mean()
    slow_ema = data['Close'].ewm(span=slow_period, adjust=False).mean()
    ppo = (fast_ema - slow_ema) / slow_ema * 100
    return ppo

data['PPO_12_26'] = calculate_ppo(data)

# Calculate ROC
def calculate_roc(data, period):
    n_days_ago = data['Close'].shift(period)
    roc = ((data['Close'] - n_days_ago) / n_days_ago) * 100
    return roc

data['ROC_14'] = calculate_roc(data, 14)

# Calculate CMFI
def calculate_cmfi(data, period):
    mfv = ((data['Close'] - data['Low']) - (data['High'] - data['Close'])) / (data['High'] - data['Low']) * data['Volume']
    cmf = mfv.rolling(window=period).sum() / data['Volume'].rolling(window=period).sum()
    return cmf

data['CMFI_20'] = calculate_cmfi(data, 20)

# Calculate DMI
def calculate_dmi(data, period):
    plus_dm = data['High'].diff()
    minus_dm = data['Low'].diff().abs()
    tr = data[['High', 'Low', 'Close']].max(axis=1) - data[['High', 'Low', 'Close']].min(axis=1)
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    plus_di = 100 * (plus_dm.rolling(window=period).sum() / tr.rolling(window=period).sum())
    minus_di = 100 * (minus_dm.rolling(window=period).sum() / tr.rolling(window=period).sum())
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di))
    adx = dx.rolling(window=period).mean()
    return adx

data['DMI_14'] = calculate_dmi(data, 14)

# Calculate PSI
def calculate_psi(data, period):
    psi = 100 * ((data['Close'] - data['Close'].shift(period)) / data['Close'].shift(period))
    return psi

data['PSI_14'] = calculate_psi(data, 14)

data = data.dropna()

# Save the updated dataframe to a new Excel file
output_file_path = file + 'WithTech.xlsx'
data.to_excel(output_file_path, index=False)

print(f"File saved as {output_file_path}")

File saved as XRPDailyWithTech.xlsx
